In [1]:
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

tf.compat.v1.enable_eager_execution()
assert tf.executing_eagerly()

In [2]:
epochs = 20
batch_size = 500

DP-SGD has three privacy-specific hyperparameters and one existing hyperamater that you must tune:

1. `l2_norm_clip` (float) - The maximum Euclidean (L2) norm of each gradient that is applied to update model parameters. This hyperparameter is used to bound the optimizer's sensitivity to individual training points. 
2. `noise_multiplier` (float) - The amount of noise sampled and added to gradients during training. Generally, more noise results in better privacy (often, but not necessarily, at the expense of lower utility).
3.   `microbatches` (int) - Each batch of data is split in smaller units called microbatches. By default, each microbatch should contain a single training example. This allows us to clip gradients on a per-example basis rather than after they have been averaged across the minibatch. This in turn decreases the (negative) effect of clipping on signal found in the gradient and typically maximizes utility. However, computational overhead can be reduced by increasing the size of microbatches to include more than one training examples. The average gradient across these multiple training examples is then clipped. The total number of examples consumed in a batch, i.e., one step of gradient descent, remains the same. The number of microbatches should evenly divide the batch size. 

In [3]:
l2_norm_clip = 0.5
noise_multiplier = 1
num_microbatches = 1

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

Percentile at which to clip norm, based on public data gradients

In [4]:
norm_percentile = 0.8

In [5]:
train, test = tf.keras.datasets.cifar10.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

assert train_data.min() == 0.
assert train_data.max() == 1.
assert test_data.min() == 0.
assert test_data.max() == 1.

In [6]:
from sklearn.model_selection import train_test_split

# test_size refers to private data size
public_data, private_data, public_labels, private_labels = \
    train_test_split(train_data, train_labels, test_size=0.98)

train_dataset = tf.data.Dataset.from_tensor_slices((private_data, private_labels))
# Data is already shuffled
train_dataset = train_dataset.batch(batch_size)
num_batches = private_data.shape[0] // batch_size

In [7]:
print(public_data.shape)
print(public_labels.shape)
print(private_data.shape)
print(private_labels.shape)

(1000, 32, 32, 3)
(1000, 10)
(49000, 32, 32, 3)
(49000, 10)


In [8]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

epsilon, _ = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    n=private_labels.shape[0], batch_size=batch_size, noise_multiplier=noise_multiplier, epochs=epochs, delta=1e-5)

DP-SGD with sampling rate = 1.02% and noise_multiplier = 1 iterated over 1960 steps satisfies differential privacy with eps = 3.38 and delta = 1e-05.
The optimal RDP order is 7.0.


(3.382117679728355, 7.0)

In [8]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    n=private_labels.shape[0], batch_size=batch_size, noise_multiplier=noise_multiplier, epochs=epochs, delta=1e-5)

DP-SGD with sampling rate = 1.02% and noise_multiplier = 1 iterated over 1960 steps satisfies differential privacy with eps = 3.38 and delta = 1e-05.
The optimal RDP order is 7.0.


(3.382117679728355, 7.0)

In [9]:
# CNN model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

def cnn_model():
    model = tf.keras.models.Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=train_data.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(train_labels.shape[1]))
    model.add(Activation('softmax'))
    
    return model

In [10]:
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPAdamGaussianOptimizer

optimizer = DPAdamGaussianOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches)

loss_fn = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [11]:
def l2_norm_grad(grad):
    summed = sum([np.sum(np.square(layer.numpy())) for layer in grad])
    return summed ** 0.5

def l2_norm_arr(arr):
    return np.sqrt(np.sum(np.square(arr)))

def get_public_grads(public_x, public_y, loss_fn, model):
    public_grads = []
    # x needs to have extra dimension for number of examples,
    # even if it's 1 for our case
    public_x = np.expand_dims(public_x, axis=1)
    for x, y in zip(public_x, public_y):
#     for x, y in tqdm(zip(public_x, public_y), total=public_x.shape[0], desc='Public Dataset Iter'):
        with tf.GradientTape() as tape:
            loss_value = loss_fn(y, model(x))
            grad = tape.gradient(loss_value, model.trainable_weights)
        public_grads.append(grad)
    return public_grads

def get_grads_percentile(public_grads, percentile):
    layer_percentiles = []
    for layer in public_grads:
        layer_percentile = np.percentile(np.stack(layer), norm_percentile, axis=0)
        layer_percentiles.append(layer_percentile)
    return layer_percentiles

def evaluate_model(model, loss_fn, x, y):
    pred = model(x)
    loss = np.mean(loss_fn(y, pred).numpy())
    acc = np.mean(tf.keras.metrics.categorical_accuracy(y, pred).numpy())
    return (loss, acc)

In [12]:
clipped_model = cnn_model()
clipped_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [ ]:
# Iterate over epochs.
clipped_loss_epochs = []
clipped_acc_epochs = []
median_norms = []

for epoch in tqdm(range(epochs), desc='Epoch'):
    
    # Evaluate
    loss, acc = evaluate_model(clipped_model, loss_fn, test_data, test_labels)
    print("Epoch %d - Loss: %.4f, Acc: %.4f" % (epoch, loss, acc))
    clipped_loss_epochs.append(loss)
    clipped_acc_epochs.append(acc)

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(tqdm(train_dataset, total=num_batches, desc='Batch')):

        public_grads = get_public_grads(public_data, public_labels, loss_fn, clipped_model)
        median_norm = np.median([l2_norm_grad(grad) for grad in public_grads])
        median_norms.append(median_norm)

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables autodifferentiation.
        with tf.GradientTape(persistent=True) as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = clipped_model(x_batch_train)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss = lambda: loss_fn(y_batch_train, logits)

            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grads = optimizer.compute_gradients(loss, clipped_model.trainable_weights, gradient_tape=tape, 
                                                new_l2_norm_clip=median_norm)

        del tape

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(grads)

Epoch 0 - Loss: 2.3026, Acc: 0.1007


Instructions for updating:
Use tf.identity instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
evaluate_model(clipped_model, loss_fn, test_data, test_labels)

In [23]:
dpsgd_model = cnn_model()
dpsgd_optimizer = DPAdamGaussianOptimizer(
                        l2_norm_clip=l2_norm_clip,
                        noise_multiplier=noise_multiplier,
                        num_microbatches=num_microbatches)
dpsgd_model.compile(optimizer=dpsgd_optimizer, loss=loss_fn, metrics=['accuracy'])

In [ ]:
# Iterate over epochs.
dpsgd_loss_epochs = []
dpsgd_acc_epochs = []

for epoch in tqdm(range(epochs), desc='Epoch'):
    
    # Evaluate
    loss, acc = evaluate_model(dpsgd_model, loss_fn, test_data, test_labels)
    print("Epoch %d - Loss: %.4f, Acc: %.4f" % (epoch, loss, acc))
    dpsgd_loss_epochs.append(loss)
    dpsgd_acc_epochs.append(acc)

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(tqdm(train_dataset, total=num_batches, desc='Batch')):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables autodifferentiation.
        with tf.GradientTape(persistent=True) as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = dpsgd_model(x_batch_train)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss = lambda: loss_fn(y_batch_train, logits)

            # Use the gradient tape to automatically retrieve
            # the gradients of the trainable variables with respect to the loss.
            grads = optimizer.compute_gradients(loss, dpsgd_model.trainable_weights, gradient_tape=tape)

        del tape

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(grads)

Epoch 0 - Loss: 2.3023, Acc: 0.0994



Epoch 1 - Loss: 2.3039, Acc: 0.1001



Epoch 2 - Loss: 2.3033, Acc: 0.1222



Epoch 3 - Loss: 2.3052, Acc: 0.1042



Epoch 4 - Loss: 2.3084, Acc: 0.0950


...


Epoch 12 - Loss: 2.2920, Acc: 0.1279



Epoch 13 - Loss: 2.2883, Acc: 0.1396



Epoch 14 - Loss: 2.2843, Acc: 0.1448



Epoch 15 - Loss: 2.2867, Acc: 0.1428



Epoch 16 - Loss: 2.2928, Acc: 0.1385


In [ ]:
evaluate_model(dpsgd_model, loss_fn, test_data, test_labels)

In [ ]:
baseline_model = cnn_model()
baseline_model.compile(optimizer='adam',
                       loss=loss_fn, metrics=['accuracy'])

In [ ]:
baseline_model.fit(private_data, private_labels,
              epochs=epochs,
              batch_size=batch_size)

In [ ]:
evaluate_model(baseline_model, loss_fn, test_data, test_labels)

In [ ]:
import pandas as pd

metrics = pd.DataFrame({'clipped_loss': clipped_loss_epochs,
                        'dpsgd_loss': dpsgd_loss_epochs,
                        'clipped_acc': clipped_acc_epochs,
                        'dpsgd_acc': dpsgd_acc_epochs})
print(metrics)
print(median_norms)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.lineplot(data=metrics)
ax.set(xlabel='Epoch', ylabel='Loss/Acc', title='CIFAR10')
plt.savefig('cifar10-{}-dynamic.png'.format(norm_percentile))